In [1]:
import asyncio
import operator
import os
from langchain.chat_models import init_chat_model
from langchain_core.messages import AnyMessage, AIMessage, HumanMessage, SystemMessage
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from pydantic import BaseModel
from typing import Annotated, Dict, List, Union
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
mcp_client = None # MCPクライアントを格納するためのグローバル変数をNoneで初期化します。
tools = None
llm_with_tools = None


In [3]:
async def initialize_llm():
    """MCPクライアントとツールを初期化する"""
    global mcp_client, tools, llm_with_tools

    mcp_client = MultiServerMCPClient(
        {
            "file-system": {
                "command": "npx",
                "args": [
                    "-y",
                    "@modelcontextprotocol/server-filesystem",
                    "./"
                ],
                "transport": "stdio",
            },

            "aws-knowledge-mcp-server": {
                "url": "https://knowledge-mcp.global.api.aws",
                "transport": "streamable_http"
            }
        }
    )

    tools = await mcp_client.get_tools()

    llm_with_tools = init_chat_model(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        model_provider="bedrock_converse",
    ).bind_tools(tools)

In [4]:
class AgentState(BaseModel):
    messages: Annotated[list[AnyMessage], operator.add]

In [5]:
system_prompt = """
あなたの責務はAWSドキュメントを検索し、Markdown形式としてファイル出力することです。
- 検索後、Markdown形式に変換してください。
- 検索は最大で2回までとし、その時点での情報を出力してください。
"""


In [6]:
async def agent(state: AgentState) -> Dict[str, List[AIMessage]]:
    response = await llm_with_tools.ainvoke(
        [SystemMessage(system_prompt)] + state.messages
    )

    return {"messages": [response]}

In [9]:
def route_node(state: AgentState) -> Union[str]:
    last_message = state.messages[-1]
    if not isinstance(last_message, AIMessage):
        raise ValueError("「AIMessage」以外のメッセージです。遷移が不正な可能性があります。")
    if not last_message.tool_calls:
        return END
    return "tools"

In [10]:
await initialize_llm()

In [12]:
builder = StateGraph(AgentState)
builder.add_node("agent", agent)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "agent")
builder.add_conditional_edges(
    "agent",route_node
)
builder.add_edge("tools","agent")
graph = builder.compile(name="ReAct Agent")

In [14]:
from rich import print
question = "Bedrockで利用可能なモデルプロパイダーを教えて！"
response = await graph.ainvoke(
    {"messages":[HumanMessage(question)]}
)

print(response)

{
    'messages': [
        HumanMessage(
            content='Bedrockで利用可能なモデルプロパイダーを教えて！',
            additional_kwargs={},
            response_metadata={}
        ),
        AIMessage(
            content=[
                {
                    'type': 'text',
                    'text': 
'AWSのBedrockで利用可能なモデルプロバイダーについて調査します。Bedrockのドキュメントを検索して情報を取得します。'
                },
                {
                    'type': 'tool_use',
                    'name': 'aws___search_documentation',
                    'input': {'search_phrase': 'Amazon Bedrock available model providers', 'limit': 5},
                    'id': 'tooluse_Y8p-CH_RT_andPMUSYZJ9Q'
                }
            ],
            additional_kwargs={},
            response_metadata={
                'ResponseMetadata': {
                    'RequestId': '90a836d9-6bed-43a4-930b-d19247c5db88',
                    'HTTPStatusCode': 200,
                    'HTTPHeaders': {
                        'date': 'Sat, 15 Nov 2025 17:40:47 GMT',
                        'content-type': 'application/json',
                        'content-length': '668',
                        'connection': 'keep-alive',
                        'x-amzn-requestid': '90a836d9-6bed-43a4-930b-d19247c5db88'
                    },
                    'RetryAttempts': 0
                },
                'stopReason': 'tool_use',
                'metrics': {'latencyMs': [2188]},
                'model_name': 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'
            },
            id='run--4414882c-69a3-4090-a23c-569b7a00a306-0',
            tool_calls=[
                {
                    'name': 'aws___search_documentation',
                    'args': {'search_phrase': 'Amazon Bedrock available model providers', 'limit': 5},
                    'id': 'tooluse_Y8p-CH_RT_andPMUSYZJ9Q',
                    'type': 'tool_call'
                }
            ],
            usage_metadata={
                'input_tokens': 4858,
                'output_tokens': 129,
                'total_tokens': 4987,
                'input_token_details': {'cache_creation': 0, 'cache_read': 0}
            }
        ),
        ToolMessage(
            content='{"content":{"result":[{"rank_order":1,"url":"https://aws.amazon.com/bedrock/developer-experien
ce/","title":"Foundation Model Choice and Customization - Amazon Bedrock Developer Experience - 
AWS","context":"Amazon Bedrock is a developer experience platform offered by AWS that simplifies the process of 
working with a range of high-performing foundation models (FMs). Developers can choose from leading FMs such as 
AI21 Labs\' Jurassic, Anthropic\'s Claude, Cohere\'s Command and Embed, Meta\'s Llama 2, Stability AI\'s Stable 
Diffusion, and Amazon\'s own Titan models. Amazon Bedrock provides interactive playgrounds for various modalities 
including text, chat, and image, enabling developers to experiment with different models for their use 
cases.\\n\\nAmazon Bedrock offers Model Evaluation, which uses automatic and human evaluations to select FMs for 
specific use cases. Automatic evaluations use curated datasets and provide predefined metrics, while human 
evaluations allow developers to bring their own datasets and define custom metrics. Human evaluations can be 
performed by the developer\'s own employees or managed by AWS.\\n\\nAmazon Bedrock enables developers to privately 
customize FMs with their data using techniques like fine-tuning and continued pretraining. Fine-tuning involves 
making a copy of the base model, training it with labeled examples from Amazon Simple Storage Service (Amazon S3), 
and creating a fine-tuned model accessible only to the developer. Continued pretraining uses unlabeled datasets to 
customize the FM for a domain or industry. Both techniques create a private, customized copy of the base FM for the
developer, with data securely transferred through Amazon Virtual Private Cloud (Amazon VPC).\\n\\nConverse API is 
provided by Ama

In [11]:

async def main(): # メイン処理を行う非同期関数を定義します。
    # MCPクライアントとツールを初期化 # MCPクライアントとツールの初期化を行います。
    await initialize_llm() # initialize_llm関数を呼び出してMCPクライアントとツール、LLMを初期化します。
    
    # グラフを構築 # LangGraphのグラフを構築します。
    builder = StateGraph(AgentState) # AgentStateを状態として持つStateGraphのビルダーを作成します。
    builder.add_node("agent", agent) # "agent"という名前でagent関数をノードとして追加します。
    builder.add_node("tools", ToolNode(tools)) # "tools"という名前でToolNodeにツールをバインドしてノードとして追加します。
    
    builder.add_edge(START, "agent") # グラフの開始（START）から"agent"ノードへのエッジを追加します。
    builder.add_conditional_edges( # 条件付きエッジを追加します。
        "agent", # "agent"ノードから条件付き遷移を開始します。
        route_node, # route_node関数を使って次のノードを決定します。
    )
    builder.add_edge("tools", "agent") # "tools"ノードから"agent"ノードへのエッジを追加します。
    
    graph = builder.compile(name="ReAct Agent") # グラフをコンパイルし、"ReAct Agent"という名前を付けます。
    
    question = "Bedrockで利用可能なモデルプロパイダーを教えて！" # エージェントに尋ねる質問を定義します。
    response = await graph.ainvoke( # グラフを非同期で実行し、質問に対する応答を取得します。
        {"messages": # メッセージを含む辞書を渡します。
            [HumanMessage(question)] # 質問をHumanMessageとしてメッセージリストに追加します。
        }
    )
    print(response) # エージェントからの応答を表示します。
    return response # 応答を返します。

asyncio.run(main()) # main関数を非同期で実行します。

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:

mcp_client = None # MCPクライアントを格納するためのグローバル変数をNoneで初期化します。
tools = None # ツールを格納するためのグローバル変数をNoneで初期化します。
llm_with_tools = None # ツールにバインドされたLLMを格納するためのグローバル変数をNoneで初期化します。

async def initialize_llm(): # LLMを初期化するための非同期関数を定義します。
    """MCPクライアントとツールを初期化する""" # 関数のdocstringです。
    global mcp_client, tools, llm_with_tools # グローバル変数を関数内で使用することを宣言します。
    
    mcp_client = MultiServerMCPClient( # MultiServerMCPClientのインスタンスを作成します。
        { # MCPサーバーの設定を辞書形式で定義します。
            # FileSystem MCPサーバー # ファイルシステムMCPサーバーの設定を開始します。
            "file-system": { # 「file-system」という名前のサーバーを設定します。
                "command": "npx", # 実行するコマンドはnpxです。
                "args": [ # npxコマンドに渡す引数をリストで定義します。
                    "-y", # npxのプロンプトに自動的にyesと答えるための引数です。
                    "@modelcontextprotocol/server-filesystem", # 実行するパッケージ名です。
                    "./" # ファイルシステムサーバーがカレントディレクトリを対象とすることを示します。
                ],
                "transport": "stdio", # サーバーとの通信に標準入出力（stdio）を使用します。
            },
            # AWS Knowledge MCPサーバー # AWSナレッジMCPサーバーの設定を開始します。
            "aws-knowledge-mcp-server": { # 「aws-knowledge-mcp-server」という名前のサーバーを設定します。
                "url": "https://knowledge-mcp.global.api.aws", # サーバーのエンドポイントURLです。
                "transport": "streamable_http", # サーバーとの通信にストリーミング可能なHTTPを使用します。
            }
        }
    )
    # MCPサーバーをLangChainツールとして取得 # 取得したMCPサーバーをLangChainのツールとして利用します。
    tools = await mcp_client.get_tools() # MCPクライアントからツールを取得し、グローバル変数toolsに格納します。
    
    # LLMの初期化 # 大規模言語モデル（LLM）の初期化を行います。
    llm_with_tools = init_chat_model( # LangChainのチャットモデルを初期化します。
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0", # 使用するモデルのIDを指定します。
        model_provider="bedrock_converse", # モデルプロバイダーとしてBedrock Converseを指定します。
    ).bind_tools(tools) # 初期化されたLLMに取得したツールをバインドします。


# ステートの定義 # エージェントの状態を定義します。
class AgentState(BaseModel): # PydanticのBaseModelを継承してAgentStateクラスを定義します。
    messages: Annotated[list[AnyMessage], operator.add] # メッセージのリストを定義し、新しいメッセージが追加されると結合されるようにします。


system_prompt = """ # LLMに与えるシステムプロンプトを定義します。
あなたの責務はAWSドキュメントを検索し、Markdown形式としてファイル出力することです。 # エージェントの主な責務を説明します。
- 検索後、Markdown形式に変換してください。 # 検索結果をMarkdown形式に変換するよう指示します。
- 検索は最大で2回までとし、その時点での情報を出力してください。 # 検索回数を最大2回に制限し、その時点での情報を出力するよう指示します。
"""
async def agent(state: AgentState) -> Dict[str, List[AIMessage]]: # エージェントの主要なロジックを実装する非同期関数を定義します。
    response = await llm_with_tools.ainvoke( # ツールにバインドされたLLMを非同期で呼び出します。
       [SystemMessage(system_prompt)] + state.messages # システムプロンプトと現在のメッセージ履歴を結合してLLMに渡します。
    )

    return {"messages": [response]} # LLMからの応答をメッセージリストとして返します。

# ルーティング関数：toolsノードかENDノードへ遷移する # グラフの次のステップを決定するルーティング関数です。
def route_node(state: AgentState) -> Union[str]: # エージェントの状態を受け取り、次に遷移するノード名を文字列で返します。
    last_message = state.messages[-1] # メッセージ履歴の最後のメッセージを取得します。
    if not isinstance(last_message, AIMessage): # 最後のメッセージがAIMessage型でない場合、エラーを発生させます。
	    raise ValueError("「AIMessage」以外のメッセージです。遷移が不正な可能性があります。") # 不正なメッセージタイプに対するエラーメッセージです。
    if not last_message.tool_calls: # 最後のAIメッセージにツール呼び出しが含まれていない場合、
        return END # ENDノードへ遷移します。
    return "tools" # ツール呼び出しがある場合は、"tools"ノードへ遷移します。

async def main(): # メイン処理を行う非同期関数を定義します。
    # MCPクライアントとツールを初期化 # MCPクライアントとツールの初期化を行います。
    await initialize_llm() # initialize_llm関数を呼び出してMCPクライアントとツール、LLMを初期化します。
    
    # グラフを構築 # LangGraphのグラフを構築します。
    builder = StateGraph(AgentState) # AgentStateを状態として持つStateGraphのビルダーを作成します。
    builder.add_node("agent", agent) # "agent"という名前でagent関数をノードとして追加します。
    builder.add_node("tools", ToolNode(tools)) # "tools"という名前でToolNodeにツールをバインドしてノードとして追加します。
    
    builder.add_edge(START, "agent") # グラフの開始（START）から"agent"ノードへのエッジを追加します。
    builder.add_conditional_edges( # 条件付きエッジを追加します。
        "agent", # "agent"ノードから条件付き遷移を開始します。
        route_node, # route_node関数を使って次のノードを決定します。
    )
    builder.add_edge("tools", "agent") # "tools"ノードから"agent"ノードへのエッジを追加します。
    
    graph = builder.compile(name="ReAct Agent") # グラフをコンパイルし、"ReAct Agent"という名前を付けます。
    
    question = "Bedrockで利用可能なモデルプロパイダーを教えて！" # エージェントに尋ねる質問を定義します。
    response = await graph.ainvoke( # グラフを非同期で実行し、質問に対する応答を取得します。
        {"messages": # メッセージを含む辞書を渡します。
            [HumanMessage(question)] # 質問をHumanMessageとしてメッセージリストに追加します。
        }
    )
    print(response) # エージェントからの応答を表示します。
    return response # 応答を返します。

asyncio.run(main()) # main関数を非同期で実行します。